# Nonlinear ADM

$$
\begin{align}
\partial_t \gamma_{ij} &= -2 \alpha K_{ij} + \mathcal{L}_\beta \gamma_{ij} \\
\partial_t K_{ij} &= -D_i D_j \alpha + \alpha \left( R_{ij} + K K_{ij} - 2 K_{ik} K^k_j \right) + \mathcal{L}_\beta K_{ij} \\
\end{align}
$$
The ricci tensor is built upon the Christoffel symbol of the second kind
$$
\Gamma^k_{ij} = \frac{1}{2} \gamma^{kl} \left( \partial_i \gamma_{jl} + \partial_j \gamma_{il} - \partial_l \gamma_{ij} \right)
$$
handled in the following way:
$$
\begin{align}
R_{ij} &= \partial_k \Gamma^k_{ij} - \partial_j \Gamma^k_{ik} + \Gamma^k_{ij} \Gamma^l_{kl} - \Gamma^k_{il} \Gamma^l_{jk} \\
\end{align}
$$
Now we create the four tensor:

$$
\begin{align}
\partial_\mu \Gamma^\nu_{ij} &= \partial_\mu \left( \frac{1}{2} \gamma^{\nu l} \left( \partial_i \gamma_{jl} + \partial_j \gamma_{il} - \partial_l \gamma_{ij} \right) \right) \\
&= \frac{1}{2} \left( \partial_\mu \gamma^{\nu l} \left( \partial_i \gamma_{jl} + \partial_j \gamma_{il} - \partial_l \gamma_{ij} \right) + \gamma^{\nu l} \partial_\mu \left( \partial_i \gamma_{jl} + \partial_j \gamma_{il} - \partial_l \gamma_{ij} \right) \right) \\
&= \frac{1}{2} \left( \partial_\mu \gamma^{\nu l} \left( \partial_i \gamma_{jl} + \partial_j \gamma_{il} - \partial_l \gamma_{ij} \right) + \gamma^{\nu l} \left( \partial_\mu \partial_i \gamma_{jl} + \partial_\mu \partial_j \gamma_{il} - \partial_\mu \partial_l \gamma_{ij} \right) \right) \\
&= \frac{1}{2} \left( \partial_\mu \gamma^{\nu l} \left( \partial_i \gamma_{jl} + \partial_j \gamma_{il} - \partial_l \gamma_{ij} \right) + \gamma^{\nu l} \left( \partial_i \partial_\mu \gamma_{jl} + \partial_j \partial_\mu \gamma_{il} - \partial_l \partial_\mu \gamma_{ij} \right) \right) \\
&= \frac{1}{2} \left(\gamma^{\nu n} \partial_\mu \gamma_{n m} \gamma^{m l} \left( \partial_i \gamma_{jl} + \partial_j \gamma_{il} - \partial_l \gamma_{ij} \right) + \gamma^{\nu l} \left(\gamma_{jl,i\mu} +  \gamma_{il,j\mu} - \gamma_{ij,l\mu} \right) \right) \\
&= \left(\gamma^{\nu n} \partial_\mu \gamma_{n m} \Gamma^m_{ij} + \frac{1}{2} \gamma^{\nu l} \left(\gamma_{jl,i\mu} +  \gamma_{il,j\mu} - \gamma_{ij,l\mu} \right) \right) \\
\end{align}
$$
Therefore $\partial_k \Gamma^k_{ij} - \partial_i \Gamma^k_{jk}$ can be written as:
$$
\begin{align}
\partial_k \Gamma^k_{ij} - \partial_i \Gamma^k_{jk} 
&=
\left(\gamma^{k n} \partial_k \gamma_{n m} \Gamma^m_{ij} + \frac{1}{2} \gamma^{k n} \left(\gamma_{jn,ik} +  \gamma_{in,jk} - \gamma_{ij,nk} \right) \right)
-
\left(\gamma^{k n} \partial_i \gamma_{n m} \Gamma^m_{jk} + \frac{1}{2} \gamma^{k n} \left(\gamma_{jn,ki} + \gamma_{kn,ji} - \gamma_{jk,ni} \right) \right)  \\
&=
\gamma^{k n} \left(
\partial_k \gamma_{n m} \Gamma^m_{ij} - \partial_i \gamma_{n m} \Gamma^m_{jk} + \frac{1}{2} \left(\gamma_{jn,ik} +  \gamma_{in,jk} - \gamma_{ij,nk} - \gamma_{jn,ki} - \gamma_{kn,ji} + \gamma_{jk,ni} \right)
\right) \\
&=
\gamma^{k n} \left(
\partial_k \gamma_{n m} \Gamma^m_{ij} - \partial_i \gamma_{n m} \Gamma^m_{jk} + \frac{1}{2} \left( +  \gamma_{in,jk} - \gamma_{ij,nk}  - \gamma_{kn,ji} + \gamma_{jk,ni} \right)
\right) \\
\end{align}
$$


In [9]:
from ngsolve import *
from zenith import * 
from ngsolve import *
from netgen.csg import *
import scipy.sparse as sp
import matplotlib.pylab as plt



# import netgen.webgui  
# from ngsolve.webgui import Draw
import netgen.gui
%gui tk

In [10]:
# standard mesh
with TaskManager():
    # create a mesh
    h = 0.3
    r = 0.8
    H = 1
    R = 3
    order = 2
    kwargs = {"bonus_intorder": 10, "inverse": "pardiso", "order": order, "mesh_order": 1}

    # BH1
    pos1 =  (1,0,0)
    lin1 = (0,1,0)
    ang1 = (0,0,1)
    mass1 =1
    BH1 = BlackHole(mass1, pos1, lin1, ang1 )

    # BH2
    pos2 =  (-1,0,0)
    lin2 = (0,-1,0)
    ang2 = (0,0,1)
    mass2 =1
    BH2 = BlackHole(mass2, pos2, lin2, ang2 )

    BHs = [BH1, BH2]

    mesh = MeshBlackHoles(BHs, h=h, R=R, H = H, r= r,  curve_order = 2)

#Draw(mesh, clipping= {"z":-1})

In [11]:
scl_h1 = H1(mesh, order=order)#, dirichlet="outer")
vec_h1 = VectorValued(scl_h1, 3)
mat_h1 = VectorValued(scl_h1, 9)

scl, dscl = scl_h1.TnT()
v, dv = vec_h1.TnT()
m, dm = mat_h1.TnT()

# define the evolution variables
gf_g     = GridFunction(mat_h1)
gf_K     = GridFunction(mat_h1)
gf_alpha     = GridFunction(scl_h1)
gf_beta     = GridFunction(vec_h1)

#es = fem.Einsum(",i->i", gf_alpha, gf_beta)

#print(es)

In [12]:
fes = gf_g.space * gf_K.space * gf_alpha.space * gf_beta.space

In [13]:
# set a positive inital condition all the variables
with TaskManager():
    def Peak(pnt = (0,0,0) ,**kwargs) : 
        return exp(-kwargs.get("sigma", 1)* ( (x-pnt[0])**2 + (y-pnt[1])**2 + (z-pnt[2])**2 ) )
    pnt1 = (-1, 0 ,0)
    pnt2 = (1, 0 ,0)
    pnt0 = (0, 0 ,0)
    
    peak1 = Peak(pnt = pnt1)
    peak2 = Peak(pnt = pnt2)
    peak0 = Peak(pnt = pnt0)
    
    bio = 10
    
    gf_g.Set( (peak0,0,0, 0,peak0, 0 , 0,0 , peak0 )  , bonus_intorder = bio)
    gf_K.Set( (0,0,0, 0,0, 0 , 0,0 , 0 )  , bonus_intorder = bio)
    gf_alpha.Set( CF((peak0))  , bonus_intorder = bio)
    gf_beta.Set( CF((0,0,0))  , bonus_intorder = bio)
    
    
    

In [14]:
#gf_out = GridFunction(fes)
gf = GridFunction(fes)

# work only with components 0, 5
gf.components[0].vec.data = gf_g.vec
gf.components[1].vec.data = gf_K.vec
gf.components[2].vec.data = gf_alpha.vec
gf.components[3].vec.data = gf_beta.vec
#Draw(Norm(gf.components[0]), mesh, "g",  clipping= {"z":-1})
#Draw(Norm(gf.components[1]), mesh, "chi",  clipping= {"z":-1})
#Draw(Norm(gf.components[2]), mesh, "A",  clipping= {"z":-1})
#Draw(Norm(gf.components[3]), mesh, "K",  clipping= {"z":-1})
#Draw(Norm(f.components[4]), mesh, "G",  clipping= {"z":-1})
#Draw(Norm(gf.components[5]), mesh, "alpha",  clipping= {"z":-1})
#Draw(Norm(gf.components[6]), mesh, "beta",  clipping= {"z":-1})
#Draw(Norm(gf.components[7]), mesh, "B",  clipping= {"z":-1})
# Draw(Norm(gf.components[8]), mesh, "phi")

In [15]:
# use embedding to create inverse of whole matrix
inverse = "sparsecholesky"

with TaskManager():
    ## g
    mass_g = BilinearForm(mat_h1, symmetric=True)
    mass_g += InnerProduct(m, dm)*dx
    mass_g.Assemble()
    mass_g_inv = mass_g.mat.Inverse(inverse=inverse)
    res_g = fes.restrictions[0]
    inv_g = res_g.T@mass_g_inv@res_g

    ## K
    mass_K = BilinearForm(mat_h1, symmetric=True)
    mass_K += InnerProduct(m, dm)*dx
    mass_K.Assemble()
    mass_K_inv = mass_K.mat.Inverse(inverse=inverse)
    res_K = fes.restrictions[1]
    inv_K = res_K.T@mass_K_inv@res_K

    ## alpha
    mass_alpha = BilinearForm(scl_h1, symmetric=True)
    mass_alpha += InnerProduct(scl, dscl)*dx
    mass_alpha.Assemble()
    mass_alpha_inv = mass_alpha.mat.Inverse(inverse=inverse)
    res_alpha = fes.restrictions[2]
    inv_alpha = res_alpha.T@mass_alpha_inv@res_alpha

    ## beta
    mass_beta = BilinearForm(vec_h1, symmetric=True)
    mass_beta += InnerProduct(v, dv)*dx
    mass_beta.Assemble()
    mass_beta_inv = mass_beta.mat.Inverse(inverse=inverse)
    res_beta = fes.restrictions[3]
    inv_beta = res_beta.T@mass_beta_inv@res_beta

    





In [16]:
def Lie(gf_M, gf_b):
    b = gf_b.Reshape((3,))
    gb = grad(gf_b).Reshape((3,3))
    M = gf_M.Reshape((3,3))
    gM = grad(gf_M).Reshape((3,3,3))
    
    return fem.Einsum("ijl,l->ij", gM, b) + fem.Einsum("kj,ki->ij", M, gb) + fem.Einsum("ik,kj->ij", M, gb)

In [17]:


def Inv_met(u):
    return Inv(u.Reshape((3,3)) )



In [18]:

# define the christoffel symbols of the first kind
def CS1(gf_g):
    print( Grad(gf_g) )
    gradg = Grad(gf_g).Reshape((3,3,3))
    # Christoffel symbols of the first kind
    # cs[i,j,k] = 1/2 * (gradg[i,k,j] + gradg[k,j,i]  -gradg[i,j,k]) )

    cs = CF( 
        (
        (1/2) * (gradg[0,0,0] + gradg[0,0,0] - gradg[0,0,0]),
        (1/2) * (gradg[0,0,1] + gradg[0,1,0] - gradg[0,0,1]),
        (1/2) * (gradg[0,0,2] + gradg[0,2,0] - gradg[0,0,2]),

        (1/2) * (gradg[0,1,0] + gradg[1,0,0] - gradg[0,1,0]),
        (1/2) * (gradg[0,1,1] + gradg[1,1,0] - gradg[0,1,1]),
        (1/2) * (gradg[0,1,2] + gradg[1,2,0] - gradg[0,1,2]),

        (1/2) * (gradg[0,2,0] + gradg[2,0,0] - gradg[0,2,0]),
        (1/2) * (gradg[0,2,1] + gradg[2,1,0] - gradg[0,2,1]),
        (1/2) * (gradg[0,2,2] + gradg[2,2,0] - gradg[0,2,2]),

        (1/2) * (gradg[1,0,0] + gradg[0,0,1] - gradg[1,0,0]),
        (1/2) * (gradg[1,0,1] + gradg[0,1,1] - gradg[1,0,1]),
        (1/2) * (gradg[1,0,2] + gradg[0,2,1] - gradg[1,0,2]),

        (1/2) * (gradg[1,1,0] + gradg[1,0,1] - gradg[1,1,0]),
        (1/2) * (gradg[1,1,1] + gradg[1,1,1] - gradg[1,1,1]),
        (1/2) * (gradg[1,1,2] + gradg[1,2,1] - gradg[1,1,2]),

        (1/2) * (gradg[1,2,0] + gradg[2,0,1] - gradg[1,2,0]),
        (1/2) * (gradg[1,2,1] + gradg[2,1,1] - gradg[1,2,1]),
        (1/2) * (gradg[1,2,2] + gradg[2,2,1] - gradg[1,2,2]),

        (1/2) * (gradg[2,0,0] + gradg[0,0,2] - gradg[2,0,0]),
        (1/2) * (gradg[2,0,1] + gradg[0,1,2] - gradg[2,0,1]),
        (1/2) * (gradg[2,0,2] + gradg[0,2,2] - gradg[2,0,2]),

        (1/2) * (gradg[2,1,0] + gradg[1,0,2] - gradg[2,1,0]),
        (1/2) * (gradg[2,1,1] + gradg[1,1,2] - gradg[2,1,1]),
        (1/2) * (gradg[2,1,2] + gradg[1,2,2] - gradg[2,1,2]),

        (1/2) * (gradg[2,2,0] + gradg[2,0,2] - gradg[2,2,0]),
        (1/2) * (gradg[2,2,1] + gradg[2,1,2] - gradg[2,2,1]),
        (1/2) * (gradg[2,2,2] + gradg[2,2,2] - gradg[2,2,2])
    ), dims=(3,3,3) )    

    cs1 = 0.5*( fem.Einsum("ijk->ikj", gradg) + fem.Einsum("ijk->jki", gradg) -fem.Einsum("ijk->ijk", gradg) )
    return cs1#.Reshape((3,3,3))
    
#cs1 = CS1(gf_g)
#print(cs1)


In [19]:

def CS2(gf_g):
    cs1 = CS1(gf_g)
    ginv = Inv_met(gf_g).Reshape((3,3))
    # Christoffel symbols of the second kind
    # cs2[i,j,k] = cs1[i,j,l] * ginv[l,k]
    cs2 = CF(
        (
        cs1[0,0,0] * ginv[0,0] + cs1[0,0,1] * ginv[1,0] + cs1[0,0,2] * ginv[2,0] ,
        cs1[0,1,0] * ginv[0,0] + cs1[0,1,1] * ginv[1,0] + cs1[0,1,2] * ginv[2,0] ,
        cs1[0,2,0] * ginv[0,0] + cs1[0,2,1] * ginv[1,0] + cs1[0,2,2] * ginv[2,0] ,

        cs1[1,0,0] * ginv[0,0] + cs1[1,0,1] * ginv[1,0] + cs1[1,0,2] * ginv[2,0] ,
        cs1[1,1,0] * ginv[0,0] + cs1[1,1,1] * ginv[1,0] + cs1[1,1,2] * ginv[2,0] ,
        cs1[1,2,0] * ginv[0,0] + cs1[1,2,1] * ginv[1,0] + cs1[1,2,2] * ginv[2,0] ,

        cs1[2,0,0] * ginv[0,0] + cs1[2,0,1] * ginv[1,0] + cs1[2,0,2] * ginv[2,0] ,
        cs1[2,1,0] * ginv[0,0] + cs1[2,1,1] * ginv[1,0] + cs1[2,1,2] * ginv[2,0] ,
        cs1[2,2,0] * ginv[0,0] + cs1[2,2,1] * ginv[1,0] + cs1[2,2,2] * ginv[2,0] ,

        cs1[0,0,0] * ginv[0,1] + cs1[0,0,1] * ginv[1,1] + cs1[0,0,2] * ginv[2,1] ,
        cs1[0,1,0] * ginv[0,1] + cs1[0,1,1] * ginv[1,1] + cs1[0,1,2] * ginv[2,1] ,
        cs1[0,2,0] * ginv[0,1] + cs1[0,2,1] * ginv[1,1] + cs1[0,2,2] * ginv[2,1] ,

        cs1[1,0,0] * ginv[0,1] + cs1[1,0,1] * ginv[1,1] + cs1[1,0,2] * ginv[2,1] ,
        cs1[1,1,0] * ginv[0,1] + cs1[1,1,1] * ginv[1,1] + cs1[1,1,2] * ginv[2,1] ,
        cs1[1,2,0] * ginv[0,1] + cs1[1,2,1] * ginv[1,1] + cs1[1,2,2] * ginv[2,1] ,

        cs1[2,0,0] * ginv[0,1] + cs1[2,0,1] * ginv[1,1] + cs1[2,0,2] * ginv[2,1] ,
        cs1[2,1,0] * ginv[0,1] + cs1[2,1,1] * ginv[1,1] + cs1[2,1,2] * ginv[2,1] ,
        cs1[2,2,0] * ginv[0,1] + cs1[2,2,1] * ginv[1,1] + cs1[2,2,2] * ginv[2,1] ,

        cs1[0,0,0] * ginv[0,2] + cs1[0,0,1] * ginv[1,2] + cs1[0,0,2] * ginv[2,2] ,
        cs1[0,1,0] * ginv[0,2] + cs1[0,1,1] * ginv[1,2] + cs1[0,1,2] * ginv[2,2] ,
        cs1[0,2,0] * ginv[0,2] + cs1[0,2,1] * ginv[1,2] + cs1[0,2,2] * ginv[2,2] ,

        cs1[1,0,0] * ginv[0,2] + cs1[1,0,1] * ginv[1,2] + cs1[1,0,2] * ginv[2,2] ,
        cs1[1,1,0] * ginv[0,2] + cs1[1,1,1] * ginv[1,2] + cs1[1,1,2] * ginv[2,2] ,
        cs1[1,2,0] * ginv[0,2] + cs1[1,2,1] * ginv[1,2] + cs1[1,2,2] * ginv[2,2] ,

        cs1[2,0,0] * ginv[0,2] + cs1[2,0,1] * ginv[1,2] + cs1[2,0,2] * ginv[2,2] ,
        cs1[2,1,0] * ginv[0,2] + cs1[2,1,1] * ginv[1,2] + cs1[2,1,2] * ginv[2,2] ,
        cs1[2,2,0] * ginv[0,2] + cs1[2,2,1] * ginv[1,2] + cs1[2,2,2] * ginv[2,2] 

        ),
        dims=(3,3,3)
    )

    cs2n = fem.Einsum("ljk,il->ijk", cs1,ginv)
    return cs2n
    

cs2 = CS2(gf_g)


coef class ngcomp::GridFunctionCoefficientFunction, real, dims = 9 x 3



$$
\begin{align}
\partial_k \Gamma^k_{ij} - \partial_i \Gamma^k_{jk} 
&=
\gamma^{k n} \left(
\partial_k \gamma_{n m} \Gamma^m_{ij} - \partial_i \gamma_{n m} \Gamma^m_{jk} + \frac{1}{2} \left( +  \gamma_{in,jk} - \gamma_{ij,nk}  - \gamma_{kn,ji} + \gamma_{jk,ni} \right)
\right) \\
\end{align}
$$

In [20]:

# def the Ricci tensor
def Ric(gf_g):
    cs2 = CS2(gf_g)
    invg = Inv_met(gf_g).Reshape((3,3))
    gradg = grad(gf_g).Reshape((3,3,3))
    hessg = gf_g.Operator("hesse").Reshape((3,3,3,3))
    # the ricci tensor is made up by 2 parts, the first involves the derivative of the christoffel symbols
    # the second involves the multiplication of the christoffel symbols. It is explained at the beginning of the code
    
    r1 = fem.Einsum("nmk,ijm->kij", gradg, cs2) - fem.Einsum("nmi,jkm->kij", gradg, cs2)
    r2 = 0.5*(fem.Einsum("injk,kn->ij", hessg, invg)  - fem.Einsum("ijnk,kn->ij", hessg, invg) \
              - fem.Einsum("knji,kn->ij", hessg, invg)  + fem.Einsum("jkni,kn->ij", hessg, invg))
    

    return fem.Einsum("kij,kn->ij",r1, invg) + r2 

def Scal(gf_g):
    Ricci = Ric(gf_g)
    inv = Inv_met(gf_g)
    return fem.Einsum("ij,ij", Ricci, inv)
    

In [21]:
def Cov_Hess(gf_alpha, gf_g):
    cs2 = CS2( gf_g)
    return gf_alpha.Operator("hesse") - cs2[0,:,:] * grad(gf_alpha)[0] - cs2[1,:,:] * grad(gf_alpha)[1] - cs2[2,:,:] * grad(gf_alpha)[2]

def Cov_Div(gf_K, gf_g):
    cs2 = CS2( gf_g)
    diver = grad(gf_K)[:, 0, 0] + grad(gf_K)[:, 1, 1] + grad(gf_K)[:, 2, 2]
    Gammacov = gf_K*cs2[0,0,:] + gf_K*cs2[1,1,:] + gf_K*cs2[2,2,:]
    Gammacontr = - gf_K*cs2[0,0,:] - gf_K*cs2[1,1,:] - gf_K*cs2[2,2,:]
    return diver + Gammacov + Gammacontr


In [22]:

# trial and test functions
g,  K, alpha, beta = fes.TrialFunction()
dg,  dK,dalpha, dbeta = fes.TestFunction()

print( alpha.Operators())

def NonlinearForm(fes):
    lf = LinearForm(fes)
    lf += -2 * gf_alpha * InnerProduct(gf_K, dg) * dx + InnerProduct(Lie(gf_g, gf_beta), dg)*dx 
    lf +=  InnerProduct(Lie(gf_K, gf_beta), dK)*dx - InnerProduct( Cov_Hess(gf_alpha, gf_g), dK)*dx
    lf +=  gf_alpha * InnerProduct( Ric(gf_g), dK)*dx 
    lf +=  gf_alpha * InnerProduct( gf_K, Inv_met(gf_g) ) * InnerProduct( gf_K, dK )*dx
    lf +=  -2* gf_alpha * InnerProduct( gf_K.Reshape((3,3))*Inv_met(gf_g).Reshape((3,3))*gf_K.Reshape((3,3)) , dK.Reshape((3,3)) )*dx
    return lf

lf = NonlinearForm(fes)
lf.Assemble()


#blf.Apply(gf_in.vec, gf_out.vec)



['Grad', 'Gradboundary', 'hesse', 'hesseboundary', 'dual']
coef class ngcomp::GridFunctionCoefficientFunction, real, dims = 9 x 3

coef class ngcomp::GridFunctionCoefficientFunction, real, dims = 9 x 3



In [23]:



blf = BilinearForm(fes)
blf += -2 * alpha * InnerProduct(K, dg) * dx + InnerProduct(Lie(g, beta), dg)*dx 
blf +=  InnerProduct(Lie(K, beta), dK)*dx - InnerProduct( Cov_Hess(alpha, g), dK)*dx
blf +=  gf_alpha * InnerProduct( Ric(g), dK)*dx 
blf +=  gf_alpha * InnerProduct( K, Inv_met(g) ) * InnerProduct( K, dK )*dx
blf +=  -2* gf_alpha * InnerProduct( K.Reshape((3,3))*Inv_met(g).Reshape((3,3))*K.Reshape((3,3)) , dK.Reshape((3,3)) )*dx
    



coef trial-function diffop = grad, real, dims = 9 x 3

coef trial-function diffop = grad, real, dims = 9 x 3



In [24]:
#A = sp.csr_matrix(blf.mat.CSR())
#plt.rcParams['figure.figsize'] = (10,10)
#plt.spy(A)
#plt.show()

In [25]:
inv_mass = inv_g +  inv_K +  inv_alpha + inv_beta 





In [27]:
dt = 0.001
t = 0
tend = 1
Draw(Norm(gf.components[0]), mesh, "g",  clipping= {"z":-1})
#scene2 = Draw(Norm(gf.components[1]), mesh, "chi",  clipping= {"z":-1})
#scene3 = Draw(Norm(gf.components[2]), mesh, "A",  clipping= {"z":-1})
#scene4 = Draw(Norm(gf.components[3]), mesh, "K",  clipping= {"z":-1})
#scene5 = Draw(Norm(gf.components[4]), mesh, "G",  clipping= {"z":-1})
#scene6 = Draw(Norm(gf.components[5]), mesh, "alpha",  clipping= {"z":-1})
#scene7 = Draw(Norm(gf.components[6]), mesh, "beta",  clipping= {"z":-1})
#scene8 = Draw(Norm(gf.components[7]), mesh, "B",  clipping= {"z":-1})

input("start")
blf.Assemble()
with TaskManager():
    while t < tend:
        
        print("t=",int(t/tend * 100),end="\r")
        
        gf.vec.data += dt*inv_mass @blf.mat *gf.vec
        #gf.vec.data += dt*inv_mass *lf.vec
        t += dt
        #Draw(Norm(gf.components[0]), mesh, "g",  clipping= {"z":-1})
        Draw(Norm(gf.components[1]), mesh, "chi",  clipping= {"z":-1})
        #scene2.Redraw()
        #scene3.Redraw()
        #scene4.Redraw()
        #scene5.Redraw()
        #scene6.Redraw()
        #scene7.Redraw()
        #scene8.Redraw()


    


In [ ]:
Draw(Norm(gf.components[0]), mesh, "g",  clipping= {"z":-1})
Draw(Norm(gf.components[1]), mesh, "K",  clipping= {"z":-1})
Draw(Norm(gf.components[2]), mesh, "alpha",  clipping= {"z":-1})
Draw(Norm(gf.components[3]), mesh, "beta",  clipping= {"z":-1})
